# Experiment Workbench

In [1]:
import os, sys, math, datetime
import pathlib
from pathlib import Path
import numpy as np
import random
from matplotlib import pyplot as plt
import PIL
import PIL.Image
import seaborn as sns

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D,DepthwiseConv2D, MaxPooling2D, AvgPool2D, GlobalAveragePooling2D, BatchNormalization, Concatenate
from tensorflow.keras.layers import ReLU
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
 
# Import the necessary MLTK APIs
from mltk.core import view_model, summarize_model, profile_model

# import workbench.config.config
from workbench.config.config import initialize
from workbench.utils.utils import create_filepaths
from workbench.utils.utils import parse_model_name

#from dotenv import load_dotenv
import wandb
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint
#import deeplake


In [2]:
print("Tensorflow version " + tf.__version__)
AUTOTUNE = tf.data.AUTOTUNE

# Confirm that TensorFlow can access GPU
device_name = tf.test.gpu_device_name()
if not device_name:
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Tensorflow version 2.10.0
Found GPU at: /device:GPU:0


In [3]:
# TODO:

# Add learning rate schedule

# tf.compat.v1.disable_eager_execution()

In [4]:
# DANGER ZONE: Disable warning messages

import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)

In [5]:
tf.keras.backend.clear_session()

https://www.tensorflow.org/api_docs/python/tf/random/set_seed  

Operations that rely on a random seed actually derive it from two seeds: the global and operation-level seeds. This sets the global seed.

Its interactions with operation-level seeds is as follows:

1. If neither the global seed nor the operation seed is set: A randomly picked seed is used for this op.  
2. If the global seed is set, but the operation seed is not: The system deterministically picks an operation seed in conjunction with the global seed so that it gets a unique random sequence. Within the same version of tensorflow and user code, this sequence is deterministic. However across different versions, this sequence might change. If the code depends on particular seeds to work, specify both global and operation-level seeds explicitly.  
3. If the operation seed is set, but the global seed is not set: A default global seed and the specified operation seed are used to determine the random sequence.  
4. If both the global and the operation seed are set: Both seeds are used in conjunction to determine the random sequence.

In [6]:
seed_1 = 1
seed_2 = 15
seed_3 = 30
seed_4 = 42
seed_5 = 75

seed = seed_1

# set the random seeds
os.environ["TF_CUDNN_DETERMINISTIC"]= "1"
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed) # setting tensorflow global seed

In [7]:
models_dir = initialize()

# Get the model

In [8]:
global model_name
#model_name = "efficientNetB0_1_96_c3_o3_keras"
#model_name = "mobilenetv1_0.1_96_c3_o3_keras"
model_name = "shufflenetv1_0.2_96_c3_o3_g1"
#model_name = "MobilenetV3small_1_96_c3_o3_keras"#, "MobilenetV3large_1_224_c3_o3_keras"# ,


In [9]:
models_path, models_summary_path, models_image_path, models_layer_df_path, models_tf_path, models_tflite_path, models_tflite_opt_path = create_filepaths(model_name)

i:\tinyml\tiny_cnn\models


In [10]:
model = keras.models.load_model(models_tf_path)

In [30]:
global base_model_name
global alpha
global resolution
global channels
global classes
global variation
global early_stopping_patience

In [31]:
base_model_name, alpha, resolution, channels, classes, variation = model_name.split("_")

In [32]:
alpha = float(alpha)
resolution = int(resolution)
classes = int(classes.strip("o"))

In [33]:
# Start a Tensorboard session
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [34]:
#os.environ['WANDB_NOTEBOOK_NAME'] = 'Experiment Workbench'

IMG_HEIGHT = resolution
IMG_WIDTH = resolution
BATCH_SIZE = 32
EPOCHS = 30
#LOGGING_STEPS = 64
MOMENTUM = 0.9
LR = 0.001
DROPOUT = 0.2
early_stopping_patience = 30

PROJECT = base_model_name
#PROJECT = "tiny_cnn troubleshooting"

shuffle_seed = seed

# Prepare the Lemon Quality Dataset

In [35]:
dataset_path = Path.cwd().joinpath("datasets", "lemon_dataset")
dataset_path.exists()

True

In [36]:
dataset_path

WindowsPath('i:/tinyml/tiny_cnn/datasets/lemon_dataset')

In [37]:
def get_lemon_quality_dataset(dataset_path, img_width, img_height, batch_size, channels, normalize=True):
    """ Fetches the lemon quality dataset and prints dataset info. It normalizes the image data to range [0,1] by default.

    Args: 
        dataset_path (Path): the file location of the dataset. Subfolders "train", "test", and "val" are expected.
        normalize (boolean): Normalizes the image data to range [0, 1]. Default: True

    Returns:
        (train_ds, val_ds, test_ds, class_names) (tuple(tf.datasets)): Tensorflow datasets for train, validation and test.
    
    """
    if dataset_path.exists():
        try:
            train_dir = dataset_path.joinpath("train")
            val_dir = dataset_path.joinpath( "val")
            test_dir = dataset_path.joinpath( "test")
        except:
            print(f"Please check the folder structure of {dataset_path}.")
            raise

    channels = int(channels.strip("c"))
    if channels==1:
        color_mode = "grayscale"
    else:
        color_mode = "rgb" 
    print(f"Color mode: {color_mode}")

    # create the labels list to avoid inclusion of .ipynb checkpoints
    #labels = ["bad_quality", "empty_background", "good_quality"]

    print("Preparing training dataset...")        
    train_ds = tf.keras.utils.image_dataset_from_directory(
        train_dir,
        subset=None,
        seed=shuffle_seed,
        image_size=((img_height, img_width)),
        #labels=labels,
        batch_size=batch_size,
        #color_mode=color_mode,
        shuffle=True
        )
    

    class_names = train_ds.class_names


    print("Preparing validation dataset...")    
    val_ds = tf.keras.utils.image_dataset_from_directory(
        val_dir,
        subset=None,
        seed=shuffle_seed,
        image_size=(img_height, img_width),
        batch_size=batch_size,
        #color_mode=color_mode,
        shuffle=True
        )
    

    print("Preparing test dataset...")    
    test_ds = tf.keras.utils.image_dataset_from_directory(
        test_dir,
        subset=None,
        seed=shuffle_seed,
        image_size=(img_height, img_width),
        batch_size=1,
        #color_mode=color_mode,
        shuffle=False
        )
    
    # Create a data augmentation stage with horizontal flipping, rotations, zooms
    data_augmentation = keras.Sequential(
        [
            tf.keras.layers.RandomFlip("horizontal"),
            tf.keras.layers.RandomRotation(0.1),
            tf.keras.layers.RandomZoom(0.1),
        ]
        )

    #train_ds= train_ds.map(lambda x, y: (data_augmentation(x), y), num_parallel_calls=tf.data.AUTOTUNE )

    
    # Normalize the data to the range [0, 1]
    if normalize:
        normalization_layer = tf.keras.layers.Rescaling(1./255, offset=-1)

        train_ds= train_ds.map(lambda x, y: (normalization_layer(x), y), num_parallel_calls=tf.data.AUTOTUNE)
        val_ds= val_ds.map(lambda x, y: (normalization_layer(x), y), num_parallel_calls=tf.data.AUTOTUNE)
        test_ds= test_ds.map(lambda x, y: (normalization_layer(x), y)) #, num_parallel_calls=tf.data.AUTOTUNE)
    else:
        pass

    print (f"Class names: {class_names}")
    print(f"Train: {train_ds.element_spec}")
    print(f"Normalize: {normalize}")
    return (train_ds, val_ds, test_ds, class_names)

In [38]:
#channels

In [39]:
train_ds, val_ds, test_ds, labels = get_lemon_quality_dataset(dataset_path, IMG_WIDTH, IMG_HEIGHT, BATCH_SIZE, channels)

Color mode: rgb
Preparing training dataset...
Found 2021 files belonging to 3 classes.
Preparing validation dataset...
Found 252 files belonging to 3 classes.
Preparing test dataset...
Found 255 files belonging to 3 classes.
Class names: ['bad_quality', 'empty_background', 'good_quality']
Train: (TensorSpec(shape=(None, 96, 96, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))
Normalize: True


In [40]:
#test_ds.element_spec

In [41]:
# class_names = labels
# print(class_names)

# plt.figure(figsize=(10, 10))
# for images, labels in train_ds.take(1):
#   for i in range(9):
#     ax = plt.subplot(3, 3, i + 1)
#     plt.imshow(images[i].numpy().astype("uint8"))
#     plt.title(class_names[labels[i]])
#     plt.axis("off")'

In [42]:
train_ds

<ParallelMapDataset element_spec=(TensorSpec(shape=(None, 96, 96, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [43]:
# 'test_ds

In [44]:
# for image_batch, labels_batch in train_ds:
#     print(image_batch.shape)
#     print(labels_batch.shape)
#     print(labels_batch)
#     break

In [45]:
# classes = len(labels)
# print(f"The dataset contains {classes } classes.")

# Define the model

In [46]:
logdir = os.path.join("logs", model_name, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
root_logdir = os.getcwd()

In [47]:
#wandb.__version__

In [48]:
tf.keras.backend.clear_session()

# optimize the data flow
AUTOTUNE = tf.data.AUTOTUNE
#train_ds = train_ds.cache().prefetch(AUTOTUNE)
train_ds = train_ds.prefetch(AUTOTUNE)
val_ds = val_ds.cache().prefetch(AUTOTUNE)

In [49]:
#api = wandb.Api()
api = wandb.Api(timeout=19)

In [50]:
# code taken from https://www.tensorflow.org/guide/keras/custom_callback#examples_of_keras_callback_applications

class EarlyStoppingAtMinLoss(keras.callbacks.Callback):
    """Stop training when the loss is at its min, i.e. the loss stops decreasing.

  Arguments:
      patience: Number of epochs to wait after min has been hit. After this
      number of no improvement, training stops.
  """

    def __init__(self, patience=0):
        super(EarlyStoppingAtMinLoss, self).__init__()
        self.patience = patience
        # best_weights to store the weights at which the minimum loss occurs.
        self.best_weights = None

    def on_train_begin(self, logs=None):
        # The number of epoch it has waited when loss is no longer minimum.
        self.wait = 0
        # The epoch the training stops at.
        self.stopped_epoch = 0
        # Initialize the best as infinity.
        self.best = np.Inf

    def on_epoch_end(self, epoch, logs=None):
        current = logs.get("loss")
        if np.less(current, self.best):
            self.best = current
            self.wait = 0
            # Record the best weights if current results is better (less).
            self.best_weights = self.model.get_weights()
        else:
            self.wait += 1
            if self.wait >= self.patience:
                self.stopped_epoch = epoch
                self.model.stop_training = True
                print("Restoring model weights from the end of the best epoch.")
                self.model.set_weights(self.best_weights)

    def on_train_end(self, logs=None):
        if self.stopped_epoch > 0:
            print("Epoch %05d: early stopping" % (self.stopped_epoch + 1))


In [51]:
class EarlyStoppingAtMaxValAccuracy(keras.callbacks.Callback):
    """Stop training when the loss is at its min, i.e. the loss stops decreasing.

  Arguments:
      patience: Number of epochs to wait after max has been hit. After this
      number of no improvement, training stops.
  """

    def __init__(self, patience=30):
        super(EarlyStoppingAtMaxValAccuracy, self).__init__()
        self.patience = patience
        # best_weights to store the weights at which the minimum loss occurs.
        self.best_weights = None

    def on_train_begin(self, logs=None):
        # The number of epoch it has waited when loss is no longer minimum.
        self.wait = 0
        # The epoch the training stops at.
        self.stopped_epoch = 0
        # Initialize the best as infinity.
        self.best = 0
        self.best_epoch = 0
        self.best_epoch_loss = np.Infinity

    def on_epoch_end(self, epoch, logs=None):
        current = logs.get("val_accuracy")
        if np.greater(current, self.best):
            self.best = current
            self.best_epoch = epoch
            self.best_epoch_loss = logs.get("val_loss")
            self.wait = 0
            # Record the best weights if current results is better (less).
            self.best_weights = self.model.get_weights()
        else:
            self.wait += 1
            if self.wait >= self.patience:
                self.stopped_epoch = epoch
                self.model.stop_training = True
                print("Restoring model weights from the end of the best epoch.")
                self.model.set_weights(self.best_weights)

        metrics = dict()
        metrics["best_epoch"] = self.best_epoch
        metrics["best_val_accuracy"] = self.best
        metrics["best_epoch_loss"] = self.best_epoch_loss

        wandb.log(metrics)



    def on_train_end(self, logs=None):
        if self.stopped_epoch > 0:
            print("Epoch %05d: early stopping" % (self.stopped_epoch + 1))
        


In [52]:
os.environ["WANDB_MODE"] = "online"
#os.environ["WANDB_MODE"] = "offline"
def train_model_wandb(model):

        # solve issue from: https://github.com/wandb/wandb/issues/3536
        if len(wandb.patched["tensorboard"]) > 0:
                wandb.tensorboard.unpatch()
                
        # Configure Tensorboard root log directory to read the debugging information
        wandb.tensorboard.patch(root_logdir=root_logdir)
        # wandb.tensorboard.patch(root_logdir="wandb.run.dir")
        
        # Generate run ids
        id = wandb.util.generate_id()

        run = wandb.init(
                # Set the project where this run will be logged
                project=PROJECT, 
                id = id, 
                resume="allow",
                sync_tensorboard=True
                )

        # Specify the configuration variables
        config = wandb.config
        
        config.batch_size = BATCH_SIZE
        #config.dropout =DROPOUT
        config.learn_rate = LR
        config.momentum = MOMENTUM
        #config.decay = 1e-6
        config.epochs = EPOCHS
        config.classes = classes
        config.id = id
        config.seed = seed
        config.architecture = model_name
        

        # enable Tensorflow Debugging
        #tf.debugging.experimental.enable_dump_debug_info("./logs/debug", 
        #        tensor_debug_mode="FULL_HEALTH", circular_buffer_size=-1)

        optimizer = tf.keras.optimizers.SGD(learning_rate=LR, momentum=MOMENTUM)
        config.optimizer = optimizer._name

        model.compile(optimizer=optimizer,
                        loss='sparse_categorical_crossentropy',
                        metrics=['accuracy'])

        logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir= wandb.run.dir, histogram_freq=10, update_freq="epoch") #, profile_batch="10, 20")
        #tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir= logdir, histogram_freq=1)
        #wandb_callback = WandbCallback()# input_type="image", labels=labels) #, validation_data = val_ds.as_numpy_iterator())

        def lr_schedule(epoch):
                """
                Returns a custom learning rate that decreases as epochs progress.
                """
                learning_rate = LR
                if epoch > 20:
                        learning_rate = 0.0001
                tf.summary.scalar('learning rate', data=learning_rate, step=epoch)
                return learning_rate

        lr_callback = LearningRateScheduler(lr_schedule)

        best_model_path = Path(wandb.run.dir).joinpath(f"best_model")

        checkpoint = WandbModelCheckpoint(best_model_path,
                monitor="val_accuracy",
                save_best_only=True,
                save_freq="epoch")

        global early_stopping_patience
        early_stopping = EarlyStopping(monitor="val_accuracy", patience=early_stopping_patience)

        callbacks =[
                tensorboard_callback,
                lr_callback,
                #wandb_callback,
                WandbMetricsLogger(),
                checkpoint,
                #early_stopping,
                EarlyStoppingAtMaxValAccuracy()
        ]

        history = model.fit(train_ds,
                epochs=EPOCHS, 
                validation_data=val_ds, 
                callbacks=callbacks
        )

        #wandb.save("last_model.h5")




        #best_model = keras.models.load_model(best_model_path) # not needed due to "restore_best_weights=True"

        y_val_true = np.concatenate([y for x, y in val_ds], axis=0)
        y_val_pred = model.predict(val_ds).argmax(axis=1)

        y_test_true = np.concatenate([y for x, y in test_ds], axis=0)
        y_test_pred = model.predict(test_ds).argmax(axis=1)

        results = model.evaluate(test_ds, batch_size=BATCH_SIZE)
        print("test loss, test acc:", results)
        wandb.log({
                "test_loss" : results[0],
                "test_accuracy" : results[1]
        })

        # log data for the confusion matrix
        wandb.log({"conf_mat" : wandb.plot.confusion_matrix(probs=None,
                        y_true=y_test_true, preds=y_test_pred,
                        class_names=labels)})


        run.finish()
        return history, model, run.id

In [53]:
wandb.finish()
history, model, run_id = train_model_wandb(model)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: susbrock. Use `wandb login --relogin` to force relogin


wandb: WARNING When using `save_best_only`, ensure that the `filepath` argument contains formatting placeholders like `{epoch:02d}` or `{batch:02d}`. This ensures correct interpretation of the logged artifacts.


Epoch 1/30
64/64 [==============================] - ETA: 0s - loss: 0.5729 - accuracy: 0.7768INFO:tensorflow:Assets written to: i:\tinyml\tiny_cnn\wandb\run-20230121_174458-x3fk9re1\files\best_model\assets


INFO:tensorflow:Assets written to: i:\tinyml\tiny_cnn\wandb\run-20230121_174458-x3fk9re1\files\best_model\assets
wandb: Adding directory to artifact (i:\tinyml\tiny_cnn\wandb\run-20230121_174458-x3fk9re1\files\best_model)... Done. 0.1s


64/64 [==============================] - 85s 662ms/step - loss: 0.5729 - accuracy: 0.7768 - val_loss: 1.0333 - val_accuracy: 0.4444 - lr: 0.0010
Epoch 2/30
63/64 [============================>.] - ETA: 0s - loss: 0.1523 - accuracy: 0.9405

...

wandb: Adding directory to artifact (i:\tinyml\tiny_cnn\wandb\run-20230121_174458-x3fk9re1\files\best_model)... Done. 0.0s


64/64 [==============================] - 5s 72ms/step - loss: 0.0165 - accuracy: 0.9960 - val_loss: 0.0432 - val_accuracy: 0.9881 - lr: 1.0000e-04
Epoch 30/30
64/64 [==============================] - ETA: 0s - loss: 0.0116 - accuracy: 0.9975

wandb: Adding directory to artifact (i:\tinyml\tiny_cnn\wandb\run-20230121_174458-x3fk9re1\files\best_model)... Done. 0.0s


255/255 [==============================] - 5s 19ms/step - loss: 0.0541 - accuracy: 0.9804
test loss, test acc: [0.05414583161473274, 0.9803921580314636]


accuracy,▁▆▇▇▇▇▇█▇████████████▇████████
best_epoch,▁▁▁▁▃▄▄▅▅▆▆▇▇█████████████████
best_epoch_loss,████▇▆▅▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
best_val_accuracy,▁▁▁▁▂▃▅▇▇█████████████████████
epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
loss,█▃▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁
lr,█████████████████████▁▁▁▁▁▁▁▁▁
test_accuracy,▁
test_loss,▁
val_accuracy,▁▁▁▁▂▃▅▇▇███▅█▇█▅▅▆█▅█▇███████
val_loss,▆▆▆▆▆▅▄▃▂▁▁▁▄▁▂▁▇▅▃▁█▁▁▁▁▁▁▁▁▁


In [35]:
#!wandb sync i:\tinyml\tiny_cnn\wandb\offline-run-20221227_091238-1vzrst0a

# Conversion to TFLite

In [55]:
# Convert the model to the TensorFlow Lite format without quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
# converter = tf.lite.TFLiteConverter.from_saved_model(models_path)
tflite_model = converter.convert()

# Save the model.
with open(models_tflite_trained_path, "wb") as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\Susanne\AppData\Local\Temp\tmpv4x7m00v\assets


INFO:tensorflow:Assets written to: C:\Users\Susanne\AppData\Local\Temp\tmpv4x7m00v\assets


# Conversion to TFLite with INT8 quantization

In [37]:
# sample_iter = test_ds.as_numpy_iterator()

# for i in range(1):
#     sample = next(sample_iter)[0]
# print("Number of samples: {}".format(sample.shape[0]))

In [38]:
# def representative_data_gen():
#     for i in range(100):
#       yield([test_ds[i].reshape(1, 1)])

In [39]:
# representative_data_gen()

In [63]:
repr_ds = test_ds.unbatch()

def representative_data_gen():
  for i_value, o_value in repr_ds.batch(1).take(48):
    yield [i_value]


# def representative_data_gen():
#     for i in range(100):
#       yield([test_ds[i].reshape(1, 1)])

# def representative_data_gen():
#   for data in test_ds.batch(1).take(100):
#     yield [tf.dtypes.cast(data, tf.float32)]

# def representative_data_gen():
#     for i in range(BATCH_SIZE):
#         yield([np.expand_dims(sample[i], axis=0)])

# testing dataset
# def representative_data_gen():
#     for _ in range(100):
#       data = np.random.rand(1, 96, 96, 3)
#       yield [data.astype(np.float32)]
 
    
converter_opt = tf.lite.TFLiteConverter.from_keras_model(model)


# set the optimization flag
converter_opt.optimizations = [tf.lite.Optimize.DEFAULT]
# enforce integer only quantization
converter_opt.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter_opt.inference_input_type = tf.uint8
converter_opt.inference_output_type = tf.uint8

# provide a representative dataset for quantization
#converter_opt.representative_dataset = tf.lite.RepresentativeDataset(representative_data_gen)
converter_opt.representative_dataset = representative_data_gen

tflite_model_opt = converter_opt.convert()

# Save the model.
with open(models_tflite_opt_path, 'wb') as f:
  f.write(tflite_model_opt)


INFO:tensorflow:Assets written to: C:\Users\Susanne\AppData\Local\Temp\tmpt3eh5j5c\assets


d:\Miniconda\envs\tiny_cnn_3\lib\site-packages\tensorflow\lite\python\convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


In [65]:
# MLTK profile model reads the mode from a path - only works for MLTK models! / Model must be trained first

profiling_results = profile_model(str(models_tflite_opt_path), accelerator=None, build=False)

Profiling model in simulator ...
Using Tensorflow-Lite Micro version: b13b48c (2022-06-08)
Searching for optimal runtime memory size ...
Determined optimal runtime memory size to be 143360


# Run the TensorFlot Lite models


In [41]:
test_image = test_ds.take(1)
test_image

<TakeDataset element_spec=(TensorSpec(shape=(None, 96, 96, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [42]:
len(test_ds)

255

In [43]:
#test_gen = test_ds.unbatch().batch(1)
test_gen = test_ds.as_numpy_iterator()
#test_gen = test_gen.next() 
#test_image = test_gen.take(1)
test_image = next(test_gen)[0]
test_image


array([[[[-0.33998156, -0.33213842, -0.35174626],
         [-0.35225183, -0.3444087 , -0.36401653],
         [-0.36940867, -0.36156553, -0.38117337],
         ...,
         [-0.3047487 , -0.30222118, -0.33359373],
         [-0.2494638 , -0.24554223, -0.27691478],
         [-0.3516084 , -0.34768683, -0.37905937]],

        [[-0.42683822, -0.42683822, -0.43468136],
         [-0.3713541 , -0.3713541 , -0.37919724],
         [-0.36905634, -0.36513478, -0.38474262],
         ...,
         [-0.30284923, -0.30284923, -0.34206492],
         [-0.29128367, -0.28344053, -0.32657778],
         [-0.29404104, -0.2861979 , -0.32933515]],

        [[-0.51554835, -0.51554835, -0.5233915 ],
         [-0.41674322, -0.41674322, -0.42458636],
         [-0.3831188 , -0.3831188 , -0.39096195],
         ...,
         [-0.3132046 , -0.3132046 , -0.35242033],
         [-0.309375  , -0.3039828 , -0.34589458],
         [-0.22161454, -0.21859676, -0.2530024 ]],

        ...,

        [[-0.7803155 , -0.7920803 , -0

In [44]:
num_test_images = len(list(test_gen))

In [45]:
test_image.shape

(1, 96, 96, 3)

In [46]:
def tflite_predict(model_path, test_image):
    # Initialize the interpreter
    interpreter = tf.lite.Interpreter(model_path=str(model_path))
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()[0]
    output_details = interpreter.get_output_details()[0]

    # Check if the input type is quantized, then rescale input data to uint8
    if input_details['dtype'] == np.uint8:  # was np.uint8
        input_scale, input_zero_point = input_details["quantization"]
        test_image = test_image / input_scale + input_zero_point
        
    test_image = test_image.astype(input_details["dtype"])
    interpreter.set_tensor(input_details["index"], test_image)
    #interpreter.set_tensor(input_details["index"], np.expand_dims(test_image[0], axis=0)) # only needed when input shape (96, 96, 3)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details["index"])[0]
    prediction = output.argmax()
    print(f"Prediction: Class {prediction} derived from {output}")

    return prediction

In [47]:
tflite_result = tflite_predict(models_tflite_opt_path, test_image)

Prediction: Class 0 derived from [255   0   0]


In [48]:

def tflite_predict_on_dataset(model_path, dataset):
    # find length of dataset
    test_gen = dataset.as_numpy_iterator()
    num_images = len(list(test_gen))

    predictions = []
    y_trues = []

    test_gen = dataset.as_numpy_iterator()
    accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
    
    # iterate over the complete test_set
    for i in range(num_images):
        test_image, y_true = next(test_gen)
        prediction = tflite_predict(model_path, test_image)
        predictions.append(prediction)
        y_trues.append(y_true[0])
        accuracy.update_state(y_true, prediction)
        print(f"{i}, {test_image.shape} - true label: {y_true[0]} vs {tflite_result}")

    #accuracy = (np.sum(predictions == y_trues) * 100) / num_images
    print(f"Accuracy: {accuracy.result()} - (Number of test samples: {num_images})")
    return predictions, y_trues    

In [49]:
preds, trues = tflite_predict_on_dataset(models_tflite_opt_path, test_ds)

Prediction: Class 0 derived from [255   0   0]
0, (1, 96, 96, 3) - true label: 0 vs 0
Prediction: Class 0 derived from [255   0   0]
1, (1, 96, 96, 3) - true label: 0 vs 0
Prediction: Class 0 derived from [255   0   1]
2, (1, 96, 96, 3) - true label: 0 vs 0
Prediction: Class 0 derived from [255   0   0]
3, (1, 96, 96, 3) - true label: 0 vs 0
Prediction: Class 0 derived from [248   0   8]
4, (1, 96, 96, 3) - true label: 0 vs 0
Prediction: Class 0 derived from [255   0   0]
5, (1, 96, 96, 3) - true label: 0 vs 0
Prediction: Class 0 derived from [255   0   0]
6, (1, 96, 96, 3) - true label: 0 vs 0
Prediction: Class 0 derived from [240   0  16]
7, (1, 96, 96, 3) - true label: 0 vs 0
Prediction: Class 0 derived from [245   0  11]
8, (1, 96, 96, 3) - true label: 0 vs 0
Prediction: Class 0 derived from [255   0   1]
9, (1, 96, 96, 3) - true label: 0 vs 0
Prediction: Class 0 derived from [255   0   0]
10, (1, 96, 96, 3) - true label: 0 vs 0
Prediction: Class 0 derived from [255   0   0]
11, (1

In [50]:
preds

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,


In [51]:
accuracy = (np.sum(preds == trues) * 100) / num_test_images
accuracy

0.0

In [52]:
# # Helper function to evaluate a TFLite model on all images
# def evaluate_model(tflite_file, model_type):
#   global test_images
#   global test_labels

#   test_image_indices = range(test_images.shape[0]) # TODO: is this correct?
#   predictions = tflite_predict(tflite_file, test_image_indices)

#   accuracy = (np.sum(test_labels== predictions) * 100) / len(test_images)

#   print('%s model accuracy is %.4f%% (Number of test samples=%d)' % (
#       model_type, accuracy, len(test_images)))

In [53]:
#input_details

In [54]:
# # code copied from: https://www.tensorflow.org/lite/performance/post_training_integer_quant

# # Helper function to run inference on a TFLite model
# def run_tflite_model(tflite_file, test_image_indices):
#   global test_images

#   # Initialize the interpreter
#   interpreter = tf.lite.Interpreter(model_path=str(tflite_file))
#   interpreter.allocate_tensors()

#   input_details = interpreter.get_input_details()[0]
#   output_details = interpreter.get_output_details()[0]


In [55]:

  # predictions = np.zeros((len(test_image_indices),), dtype=int)
  # for i, test_image_index in enumerate(test_image_indices):
  #   test_image = test_images[test_image_index]
  #   test_label = test_labels[test_image_index]

  #   # Check if the input type is quantized, then rescale input data to uint8
  #   if input_details['dtype'] == np.uint8:
  #     input_scale, input_zero_point = input_details["quantization"]
  #     test_image = test_image / input_scale + input_zero_point

  #   test_image = np.expand_dims(test_image, axis=0).astype(input_details["dtype"])
  #   interpreter.set_tensor(input_details["index"], test_image)
  #   interpreter.invoke()
  #   output = interpreter.get_tensor(output_details["index"])[0]

  #   predictions[i] = output.argmax()

  # return predictions

# Model evaluation

In [28]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(test_ds, batch_size=BATCH_SIZE)
print("test loss, test acc:", results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
# print("Generate predictions for 3 samples")
# predictions = model.predict(x_test[:3])
# print("predictions shape:", predictions.shape)

Evaluate on test data


ValueError: in user code:

    File "d:\Miniconda\envs\tiny_cnn_3\lib\site-packages\keras\engine\training.py", line 1727, in test_function  *
        return step_function(self, iterator)
    File "d:\Miniconda\envs\tiny_cnn_3\lib\site-packages\keras\engine\training.py", line 1713, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "d:\Miniconda\envs\tiny_cnn_3\lib\site-packages\keras\engine\training.py", line 1701, in run_step  **
        outputs = model.test_step(data)
    File "d:\Miniconda\envs\tiny_cnn_3\lib\site-packages\keras\engine\training.py", line 1667, in test_step
        self.compute_loss(x, y, y_pred, sample_weight)
    File "d:\Miniconda\envs\tiny_cnn_3\lib\site-packages\keras\engine\training.py", line 1052, in compute_loss
        return self.compiled_loss(
    File "d:\Miniconda\envs\tiny_cnn_3\lib\site-packages\keras\engine\compile_utils.py", line 265, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=

In [57]:
# entity = "susbrock"


# run = api.run(f"{entity}/{PROJECT}/{run_id}")
# run.summary["test_accuracy"] = results[1]
# run.summary["test_loss"] = results[0]
# run.summary.update()

In [58]:
results

[0.07146961241960526, 0.9725490212440491]

In [59]:
scores = model.evaluate(test_ds, verbose=0)

In [60]:
scores

[0.07146961241960526, 0.9725490212440491]

In [61]:
model.metrics_names

['loss', 'accuracy']

In [62]:
test_predictions = model.predict(test_ds)
test_predictions

255/255 [==============================] - 4s 14ms/step


array([[9.99985576e-01, 1.85058454e-07, 1.42443223e-05],
       [9.99472201e-01, 2.19188769e-05, 5.05886273e-04],
       [9.95712399e-01, 3.46672925e-04, 3.94088728e-03],
       [9.99792516e-01, 1.46384230e-06, 2.06044424e-04],
       [9.12287831e-01, 1.11508649e-03, 8.65971223e-02],
       [9.99769866e-01, 7.42999373e-06, 2.22730276e-04],
       [9.99756634e-01, 1.79441020e-06, 2.41542584e-04],
       [9.51880872e-01, 6.56784250e-05, 4.80534136e-02],
       [9.26768303e-01, 7.82446354e-04, 7.24493489e-02],
       [9.97744679e-01, 5.07638606e-06, 2.25019827e-03],
       [9.99657989e-01, 1.59539559e-05, 3.26103676e-04],
       [9.99945402e-01, 1.05058079e-05, 4.41570519e-05],
       [9.99861836e-01, 2.26321481e-05, 1.15518378e-04],
       [9.99972820e-01, 2.71869226e-06, 2.44858802e-05],
       [9.95697260e-01, 3.65680098e-05, 4.26622247e-03],
       [9.93911028e-01, 1.24154531e-03, 4.84736403e-03],
       [9.99712646e-01, 1.34641141e-06, 2.86010443e-04],
       [9.99807656e-01, 2.10384

In [63]:
top_pred_ids = test_predictions.argmax(axis=1)
len(top_pred_ids)

255

In [64]:
top_pred_ids

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0], dtype=int64)

In [65]:
y_true = [y for x, y in test_ds]
y_true

[<tf.Tensor: shape=(1,), dtype=int32, numpy=array([0])>,
 <tf.Tensor: shape=(1,), dtype=int32, numpy=array([0])>,
 <tf.Tensor: shape=(1,), dtype=int32, numpy=array([0])>,
 <tf.Tensor: shape=(1,), dtype=int32, numpy=array([0])>,
 <tf.Tensor: shape=(1,), dtype=int32, numpy=array([0])>,
 <tf.Tensor: shape=(1,), dtype=int32, numpy=array([0])>,
 <tf.Tensor: shape=(1,), dtype=int32, numpy=array([0])>,
 <tf.Tensor: shape=(1,), dtype=int32, numpy=array([0])>,
 <tf.Tensor: shape=(1,), dtype=int32, numpy=array([0])>,
 <tf.Tensor: shape=(1,), dtype=int32, numpy=array([0])>,
 <tf.Tensor: shape=(1,), dtype=int32, numpy=array([0])>,
 <tf.Tensor: shape=(1,), dtype=int32, numpy=array([0])>,
 <tf.Tensor: shape=(1,), dtype=int32, numpy=array([0])>,
 <tf.Tensor: shape=(1,), dtype=int32, numpy=array([0])>,
 <tf.Tensor: shape=(1,), dtype=int32, numpy=array([0])>,
 <tf.Tensor: shape=(1,), dtype=int32, numpy=array([0])>,
 <tf.Tensor: shape=(1,), dtype=int32, numpy=array([0])>,
 <tf.Tensor: shape=(1,), dtype=

In [66]:
y_true = np.concatenate([y for x, y in test_ds], axis=0)
len(y_true)

255

In [67]:
confusion_mtx = tf.math.confusion_matrix(y_true, top_pred_ids, num_classes=classes)
    # list(ds_test.map(lambda x, y: y)),
    # predict_class_label_number(test_data),
    # num_classes=len(label_names))
    
confusion_mtx

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 93,   0,   3],
       [  0,  46,   0],
       [  4,   0, 109]])>

In [68]:
# sns.heatmap(confusion_mtx, xticklabels=labels, yticklabels=labels, 
#               annot=True, fmt='g')

In [69]:
def show_confusion_matrix(cm, labels):
  plt.figure(figsize=(6, 6))
  sns.heatmap(cm, xticklabels=labels, yticklabels=labels, 
              annot=True, fmt='g')
  plt.xlabel('Prediction')
  plt.ylabel('Label')
  plt.show()

In [70]:
#show_confusion_matrix(confusion_mtx, labels)

In [71]:
top_pred_ids

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0], dtype=int64)

In [72]:
#val_predictions

## Code reserved for troubleshooting

In [73]:
def mobilenet_v1_keras(input_shape, classes=classes, alpha=alpha):
    model = tf.keras.applications.mobilenet.MobileNet(
        input_shape=input_shape,
        alpha=alpha,
        depth_multiplier=1,
        dropout=0.001,
        include_top=True,
        weights=None, #'imagenet'
        input_tensor=None,
        pooling=None,
        classes=classes,
        classifier_activation='softmax',
        #**kwargs
    )

    #model._name = model.name + "_keras" # model.name cannot be overritten

    return model
    #model = mobilenet_v1_keras((IMG_WIDTH, IMG_HEIGHT, 3), classes=classes, alpha=alpha)

In [74]:
#os.environ["WANDB_MODE"] = "online"
def train_model(model):

        # solve issue from: https://github.com/wandb/wandb/issues/3536
        # if len(wandb.patched["tensorboard"]) > 0:
        #         wandb.tensorboard.unpatch()
                
        # Configure Tensorboard root log directory to read the debugging information
        #wandb.tensorboard.patch(root_logdir=root_logdir)
        # wandb.tensorboard.patch(root_logdir="wandb.run.dir")
        
        # wandb.init(
        #         # Set the project where this run will be logged
        #         project=PROJECT, 
        #         # Track hyperparameters and run metadata
        #         #config={
        #         #"learning_rate": LR,
        #         #"epochs": EPOCHS,
        #         #},
        #         sync_tensorboard=True
        #         )



        # config = wandb.config
        # # Specify the configuration variables
        # config.batch_size = BATCH_SIZE
        # config.dropout =DROPOUT
        # config.learn_rate = LR
        # #config.decay = 1e-6
        # #config.momentum = 0.9
        # config.epochs = EPOCHS
        # config.classes = classes
        

        # enable Tensorflow Debugging
        #tf.debugging.experimental.enable_dump_debug_info("./logs/debug", 
        #        tensor_debug_mode="FULL_HEALTH", circular_buffer_size=-1)

        #model = mobilenet
        model.compile(optimizer='adam',
                        loss='sparse_categorical_crossentropy',
                        metrics=['accuracy'])

        logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
        #tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir= wandb.run.dir, histogram_freq=10, update_freq="epoch") #, profile_batch="10, 20")
        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir= logdir) #, histogram_freq=1)
        #wandb_callback = WandbCallback()# input_type="image", labels=labels) #, validation_data = val_ds.as_numpy_iterator())

        early_stopping = EarlyStopping(monitor="val_accuracy", patience= early_stopping_patience)

        #checkpoint = ModelCheckpoint("my_tiny_model", save_weights_only=True)

        callbacks =[
                #tensorboard_callback,
                #wandb_callback,
                #WandbMetricsLogger(),
                #checkpoint,
                #early_stopping
        ]

        history = model.fit(train_ds,
                epochs=EPOCHS, 
                validation_data=val_ds, 
                callbacks=callbacks
        )

        # wandb.log({
        #         "loss": history.history["loss"],
        #         "accuracy": history.history["accuracy"],
        #         "val_loss": history.history["val_loss"],
        #         "val_accuracy": history.history["val_accuracy"],                                
        # })
        
        #wandb.finish()
        return history, model


In [75]:
# model.compile(optimizer='adam',
#                 loss='sparse_categorical_crossentropy',
#                 metrics=['accuracy'])